In [ ]:
%load_ext autoreload
%autoreload 2

Declaration of parameters (you must also add a tag for this cell - parameters)

In [ ]:
# specify parameters
pipeline_params={
}
step_params={
}
substep_params={
}

In [ ]:
# define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_train", ENTITY_NAME: "yolox_obj_detector"} # stored detector files from train step
    ],
    
    tmp_entities =
    [
        { ENTITY_NAME: "yolox_obj_detector" } # temporary detector files from train step
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" } # stored BentoService
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface model_pack_interface.drawio](./imgs/model_pack_interface.drawio.png)

### Loading files of trained model from the model_train component 
(weights, configs, testing image)

In [ ]:
from sinara.store import SinaraStore

inputs = substep.inputs(step_name = "model_train")
tmp_entities = substep.tmp_entities()
# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.yolox_obj_detector, tmp_dir=tmp_entities.yolox_obj_detector)

### Reading the configuration file and adding parameters

In [ ]:
import json
import os
import os.path as osp
config_fn = os.path.join(tmp_entities.yolox_obj_detector, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)
    
DEVICE = "cuda:0"
MODEL_NAME = CONFIG["train_config_parameters"]["MODEL_NAME"]
CONFIG['work_dir'] = tmp_entities.yolox_obj_detector
WORK_DIR = CONFIG['work_dir']
CONFIG['device'] = DEVICE

test_image_path = osp.join(tmp_entities.yolox_obj_detector, "test.jpg")
assert osp.exists(test_image_path)

### Select model weights for packaging

In [ ]:
# Selecting a weights file to convert to onnx format (best, latest epoch, etc.)
model_path = tmp_entities.yolox_obj_detector
save_best  = False

files = [osp.join(model_path, file) for file in os.listdir(model_path)]
models_pth  = [file for file in files if '.pth' in file and not os.path.islink(file)]
best_models = [file for file in models_pth if 'best' in file]
latest_models = [file for file in models_pth if 'latest' in file]

assert len(best_models), 'model have not pth files.'

if save_best and len(best_models) > 0:
    models_pth = best_models
else:
    models_pth = latest_models    
models_pth.sort(key=lambda file: osp.getmtime(file))
select_model = models_pth[-1]
cfg_model = osp.join(CONFIG['work_dir'], "last_cfg.py")

# set selected weights in config
CONFIG['config_model'] = cfg_model
CONFIG['torch_model'] = select_model
CONFIG['test_image_path'] = test_image_path
with open(config_fn, 'w') as f_id:
    json.dump(CONFIG, f_id, indent=4)

### Pack binary  model

#### Preparing the model for packaging

In [ ]:
# Clean model weiths - delete data of optimizer
import torch

state_dict = torch.load(CONFIG['torch_model'], map_location=torch.device('cpu'))

print(f"{state_dict.keys()=}")
if "optimizer" in state_dict:
    del state_dict["optimizer"]
    
clean_filename = osp.splitext(CONFIG['torch_model'])[0]+"_clean.pth"
torch.save(state_dict, clean_filename)

#### Packaging of the model in bento_service

In [ ]:
from bento_service import ObjDetect_Pack

outputs = substep.outputs()

# initialize bento service
model_service = ObjDetect_Pack()
model_service.pack('model_name', MODEL_NAME)
serv_v = f"{outputs.fullname('bento_service')}.{substep.run_id}"
model_service.pack('service_version', serv_v)

# get model weights as binary file for bento_artifact
with open(clean_filename, "rb") as f_id:
    f_model = f_id.read()
    
# get model config as binary file for bento_artifact
with open(CONFIG['config_model'], "rb") as f_id:
    f_config = f_id.read()
    
model_service.pack('model', f_model)
model_service.pack('config', f_config)

# get test image as binary file for bento_artifact
with open(test_image_path, "rb") as f_id:
    f_test_image = f_id.read()
    
model_service.pack('test_image', f_test_image)

### Send packaged model to outputs

In [ ]:
# save model as a bento pack
from sinara.bentoml import save_bentoservice
save_bentoservice(model_service, path=outputs.bento_service, service_version=serv_v)